In [1]:
import json
import os
from utils import create_driver
import time, random
from tqdm import tqdm
import pandas as pd

from google_scholar import scrape_publications, search_from_google_scholar_profile

In [25]:
raw_dr_ntu_dir='./raw_dr_ntu'
raw_faculty_db = 'scse_profile'

process_faculty_db_dir ='./processed'
process_faculty_db = 'scse_profile'

raw_google_search_dir = './google_search'
os.makedirs(raw_google_search_dir, exist_ok=True)

In [3]:
faculties = pd.read_csv(os.path.join(process_faculty_db_dir, process_faculty_db+'.csv'))
missing_google_scholar= pd.DataFrame(faculties[faculties['google_scholar'].isna()])

In [4]:
# Search for profile using google scholar profile search

profile_search_url = 'https://scholar.google.com/citations?view_op=search_authors&hl=en'

driver = create_driver(debug=True)

driver.get(profile_search_url)
time.sleep(random.uniform(2,4))

search_results = {'full_name': [], 'email':[], 'results': []}
for i, row in tqdm(missing_google_scholar.iterrows()):
    name = row['full_name']

    results = search_from_google_scholar_profile(name, driver)

    search_results['results'] += results
    search_results['full_name'] += [name]*len(results)
    search_results['email'] += [row['email']]*len(results)

    time.sleep(random.uniform(1,3))
    if random.uniform(0,1)>0.75:
        time.sleep(random.uniform(3,5))

    if random.uniform(0,1)>0.8:
        driver.quit()
        driver = create_driver(debug=False)
        driver.get(profile_search_url)

driver.quit()

51it [08:21,  9.84s/it]


In [5]:
search_results_df = pd.DataFrame(search_results)
search_results_df = search_results_df.merge(faculties, how='left', on=['full_name', 'email'])

search_results_df['dr_ntu_pubs'] = None
for i, row in search_results_df.iterrows():
    
    with open(f'./{raw_dr_ntu_dir}/{row["dr_ntu_id"]}.json', 'r') as f:
        raw_dr_ntu_profile = json.load(f)
    pubs = []
    for type in ['articles', 'books','book_chapters', 'conferences']:
        pubs += raw_dr_ntu_profile[type]

    search_results_df.at[i, 'dr_ntu_pubs'] = pubs

In [6]:
search_results_df['google_publications'] = None

driver = create_driver(debug=True)
for i, row in tqdm(search_results_df.iterrows()):
    driver.get(row['results'])
    time.sleep(random.uniform(2,5))

    g_publication = scrape_publications(driver)

    search_results_df.at[i,'google_publications'] = g_publication

    if random.uniform(0,1)>0.8:
        time.sleep(random.uniform(2,5))
    
    if random.uniform(0,1)>0.9:
        driver.quit()
        driver = create_driver(debug=False)
driver.quit()

78it [14:34, 11.22s/it]


In [23]:
search_results_df['search_score'] = 0
for i, row in tqdm(search_results_df.iterrows()):
    dr_ntu_pubs = row['dr_ntu_pubs']
    g_pubs = row['google_publications']

    if len(dr_ntu_pubs)==0:
        continue

    cnt = 0
    for title in dr_ntu_pubs:
        for g_pub in g_pubs:
            if g_pub['title'].lower() in title.lower():
                cnt +=1
                break

    if cnt/len(dr_ntu_pubs)>0.0:
        search_results_df.at[i,'search_score'] = cnt/len(dr_ntu_pubs)

0it [00:00, ?it/s]/var/folders/h6/xlmxflxx3_q00qrdyppcwlk40000gn/T/ipykernel_42831/3856051135.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.8536585365853658' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  search_results_df.at[i,'search_score'] = cnt/len(dr_ntu_pubs)
78it [00:00, 1380.43it/s]


In [24]:
final_results = search_results_df.loc[search_results_df[search_results_df['search_score']>0].groupby(by=['full_name', 'email'])['search_score'].idxmax()][['full_name', 'email', 'results']]

In [28]:
final_results.to_csv(f'{raw_google_search_dir}/results.csv')

In [30]:
faculties = pd.read_csv(os.path.join(process_faculty_db_dir, process_faculty_db+'.csv'))

faculties = faculties.merge(final_results, how='left', on=['full_name', 'email'])

faculties['google_scholar'] = faculties['google_scholar'].combine_first(faculties['results'])

faculties = faculties.drop(columns=['results'])

faculties['google_scholar_id'] = faculties['google_scholar'].apply(lambda x: x.split('user=')[1].split('&')[0] if isinstance(x, str) else x)

faculties.to_csv(os.path.join(process_faculty_db_dir, process_faculty_db+'.csv'), index=False)